In [309]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

### Funciones

In [310]:
#Funcion para capturar los objetos por etiqueta y clase
def buscar_tag_por_etiqueta_clase(html, etiqueta, clase):
    soup = BeautifulSoup(html, "html.parser")
    return soup.find_all(etiqueta, class_= re.compile(clase +"$"))
 

### Clases

In [311]:
class Producto:
    pass

### Proceso

In [326]:
#usaremos request para consultar por una dirección url y obtener el contenido, en este caso leer productos de supermecado lider
ruta = "https://www.lider.cl/supermercado/search?Ntt=notebook&ost=notebook"
sitio = requests.get(ruta)

In [327]:
#Buscamos nuestros tag que contienen los productos del carro
listado_items = buscar_tag_por_etiqueta_clase(sitio.content, "div", "product-item-box")

In [328]:
#Ahora vamos a recorrer cada producto del carro, según la lista que encontramos en el paso anterior. Generamos una lista de productos donde 
#guardaremos nuestros objeto Producto, según los valores que rescatamos dentro del ciclo for

productos = []
for i in listado_items:
    producto = Producto()

    if(i["prod-number"] != ""):
        producto.producto_codigo = i["prod-number"]
        
    if (i.find('span', class_="product-description js-ellipsis", recursive=True) != []):
        producto.nombre = i.find('span', class_="product-description js-ellipsis", recursive=True).text
    
    if (i.find('span', class_="product-name", recursive=True) != []):
        producto.marca = i.find('span', class_="product-name", recursive=True).text
      
    if (i.find('span', class_="product-attribute", recursive=True) != []):
        producto.formato = i.find('span', class_="product-attribute", recursive=True).text
        
    if (i.find('span', class_="price-sell", recursive=True) != []):
        producto.valor = i.find('span', class_="price-sell", recursive=True).text
    
    if (i.find('div', class_="product-legend hidden", recursive=True) != []):
        producto.vigencia = i.find('div', class_="product-legend hidden", recursive=True).text
   
    productos.append(producto)


In [329]:
#Creamos un dataframe a partir de nuestra lista de productos capturados en el paso anterior
df = pd.DataFrame.from_records([t.__dict__ for t in productos ])
df.head(100)

,producto_codigo,nombre,marca,formato,valor,vigencia
0,1203646,Notebook 240 G8 Intel Celeron N4020 / 4GB RAM ...,HP,,$179.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
1,1278703,"Notebook Ideapad 3/14""/Intel Pentium Gold/4GB/...",Lenovo,,$279.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
2,1221668,Notebook 14-dq2025la Intel Core i3-1115G4 / 4G...,HP,,$399.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
3,1263639,"Notebook 100e Chromebook 11.6""/Intel Celeron/4...",Lenovo,,$99.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
4,1155250,Notebook IdeaPad 3 /AMD Ryzen 3 5300U/8Gb/256G...,Lenovo,,$329.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
5,1278689,"Notebook IdeaPad 5/15.6""/Intel Core i5/8 GB/51...",Lenovo,,$529.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
6,1195206,Notebook Asus Vivobook 15 OLED K513EA-L12106T ...,Asus,,$679.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
7,1142532,Notebook MacBook Notebook MacBook Air Ret 13.3...,Apple,,$999.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.
8,1279046,Stickers Notebook Multicolor,1KO,,$1.990,Vigencia del 03-8-16 al 04-8-16. Stock: 200 Un.


Finalmente tenemos como resultado un dataframe con la información de los productos que encontramos en el sitio que ingresamos como url :)